In [4]:
pip install autogen-agentchat

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip show autogen-agentchat

Name: autogen-agentchat
Version: 0.4.9.3
Summary: AutoGen agents and teams library
Home-page: 
Author: 
Author-email: 
License: MIT License
        
            Copyright (c) Microsoft Corporation.
        
            Permission is hereby granted, free of charge, to any person obtaining a copy
            of this software and associated documentation files (the "Software"), to deal
            in the Software without restriction, including without limitation the rights
            to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
            copies of the Software, and to permit persons to whom the Software is
            furnished to do so, subject to the following conditions:
        
            The above copyright notice and this permission notice shall be included in all
            copies or substantial portions of the Software.
        
            THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
            IMPLIED, INCLUDING B

In [ ]:
import os
import json
import datetime
from typing import Optional

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.ui import Console

from duckduckgo_search import DDGS

# --- Tool Functions ---

def log_event(event_details: str) -> str:
    """Saves the specified event to a log file with a timestamp."""
    log_file = "assistant_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] {event_details}\n")
        return f"Event logged successfully to {log_file}: {event_details}"
    except Exception as e:
        print(f"Error logging event: {e}")
        return f"Error logging event: {e}"

def add_scheduled_event(subject: str, attendees: list[str], time_preference: str, duration_minutes: int) -> str:
    """Saves a simulated calendar event to a separate log file."""
    log_file = "calendar_log.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    attendee_str = ", ".join(attendees)
    log_entry = f"[{timestamp}] Scheduled Event (Simulated): Subject='{subject}', Attendees='{attendee_str}', Time='{time_preference}', Duration='{duration_minutes} minutes'"

    try:
        with open(log_file, "a", encoding="utf-8") as f:
            f.write(log_entry + "\n")
        return f"Simulated event '{subject}' logged successfully to {log_file}."
    except Exception as e:
        print(f"Error logging calendar event: {e}")
        return f"Error logging simulated event: {e}"

def perform_web_search(query: str, max_results: int = 5) -> str:
    """Performs a web search using DuckDuckGo and returns formatted text results."""
    print(f"[Web Search] Searching DDG for: '{query}' (Max {max_results})") 

    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=max_results))
            if not results:
                return f"No search results found for '{query}'."
            formatted_results = "\n\n".join([f"Title: {res.get('title', 'N/A')}\nLink: {res.get('href', 'N/A')}\nSnippet: {res.get('body', 'N/A')}" for res in results])
            return f"Search results for '{query}':\n{formatted_results}"
    except Exception as e:
        print(f"Error during web search: {e}")
        return f"Error during web search for '{query}': {e}"

def save_email_draft(to: str, subject: str, body: str) -> str:
    """Saves the generated email draft to a file."""
    draft_file = "email_drafts.txt"
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    draft_content = f"""--- Draft Saved: {timestamp} ---
To: {to}
Subject: {subject}
--- Body ---
{body}
--- End Draft ---
"""
    
    try:
        with open(draft_file, "a", encoding="utf-8") as f:
            f.write(draft_content + "\n\n")
        return f"Email draft for {to} with subject '{subject}' saved to {draft_file}."
    except Exception as e:
        print(f"Error saving email draft: {e}")
        return f"Error saving email draft: {e}"

def add_todo_item(task: str) -> str:
    """Adds the given task to a to-do list file."""
    todo_file = "todo_list.txt"
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

    try:
        with open(todo_file, "a", encoding="utf-8") as f:
            f.write(f"[{timestamp}] - {task}\n")
        return f"Task '{task}' added to the To-Do list ({todo_file})."
    except Exception as e:
        print(f"Error adding todo: {e}")
        return f"Error adding task to To-Do list: {e}"

# --- LLM Client Configuration ---

config_list_path = "OAI_CONFIG_LIST.json" # Path to the JSON configuration file
try:
    with open(config_list_path, 'r', encoding='utf-8') as f:
        config_data = json.load(f)

    if isinstance(config_data, list) and len(config_data) > 0:
        llm_config_params = config_data[0]
    elif isinstance(config_data, dict):
         llm_config_params = config_data
    else:
        raise ValueError(f"No valid configuration found in {config_list_path}.")

    api_key = llm_config_params.get("api_key")
    model_name = llm_config_params.get("model")

    if not api_key:
        raise ValueError("OpenAI API key not found in JSON file.")

    model_client = OpenAIChatCompletionClient(
        model=model_name,
        api_key=api_key,
    )

except FileNotFoundError:
    print(f"Error: Configuration file not found: {config_list_path}")
    exit()
except (json.JSONDecodeError, ValueError, KeyError) as e:
    print(f"Error: Configuration error ({config_list_path}): {e}")
    exit()
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    exit()


# --- Agent Definitions ---

# Orchestrator / Supervisor Agent
orchestrator = AssistantAgent(
    name="Orchestrator",
    model_client=model_client,
    system_message="""You are the Orchestrator/Supervisor Agent managing a multi-agent team. Your goal is to fulfill the user's request by following the plan provided by Intent_Task_Router.

**Core Responsibilities:**
1.  Receive and understand the plan.
2.  Delegate tasks to the appropriate agents sequentially according to the plan.
3.  Provide necessary inputs to agents, extracting relevant information from previous steps (especially extracting ONLY snippets from web search results for summarization. Do not edit or summarize).
4.  Explicitly instruct agents when to call specific tools (`perform_web_search`, `save_email_draft`, `add_todo_item`, `add_scheduled_event`, `log_event`) with correct arguments based on the plan.
5.  Wait for and verify confirmation messages after tool calls before proceeding.
6.  Handle agent errors reasonably (e.g., retry with corrected input if possible).
7.  Ensure ALL planned steps are completed in sequence. DO NOT skip steps.
8.  Adhere strictly to agent roles; do not let agents perform tasks outside their designated function. IGNORE extra work done by agents.

**Critical Workflow Points:**
*   **Email Saving:** After `Email_Composition_Sentiment_Agent` provides the email BODY, you MUST immediately instruct it again to call `save_email_draft` with the body, recipient, and subject. Confirm the save before moving on.Do not ask to save again if already confirmed.
*   **Final Output:** After all planned steps (including final logging) are confirmed complete, formulate a concise summary of **confirmed actions** for the user, prefix it EXACTLY with "FINAL_OUTPUT: ", and end the ENTIRE message IMMEDIATELY with "TERMINATE". Example: "FINAL_OUTPUT: Search done, summary created, email draft saved, todo added, meeting logged. TERMINATE"
*   Do not send any message after the FINAL_OUTPUT/TERMINATE message.

Think step-by-step about the next action based on the plan and the last message before deciding which agent to instruct.""",
    description="Manages workflow via plan, delegates tasks/tools sequentially, extracts snippets, ensures email saving & calendar logging, handles errors, provides final summary (FINAL_OUTPUT:), then terminates."
)

# User Proxy Agent
user_proxy = UserProxyAgent(
    name="User_Proxy",
    description="Represents the end-user who initiates the request. Does not perform tasks itself, but receives the final outcome. May provide input if clarification is explicitly requested."
)

# Input Preprocessor Agent
input_preprocessor = AssistantAgent(
    name="Input_Preprocessor",
    model_client=model_client,
    system_message="""You are the Input Preprocessing Agent.
1.  Receive the raw user query from the Orchestrator.
2.  Clean grammar, spelling, and identify key entities.
3.  **Your ONLY output MUST be the cleaned query itself, prefixed with 'Cleaned Query:'.**
    Example: 'Cleaned Query: Search the web for AutoGen benefits, summarize them, email Alex, and add a todo.'
4.  DO NOT add conversational filler, explanations, or confirmations.""",
    description="Cleans and preprocesses the initial user query, returning only the cleaned text prefixed with 'Cleaned Query:'."
)

# Intent and Task Routing Agent
intent_task_router = AssistantAgent(
    name="Intent_Task_Router",
    model_client=model_client,
    system_message="""You are the Intent & Task Routing Agent.
1.  Receive the preprocessed query from the Orchestrator.
2.  Determine user intent(s).
3.  Create a sequential plan outlining the agents needed and their inputs/tasks. Use the official agent names (Web_Search_Agent, Summarization_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent, Scheduling_Calendar_Agent, Analytics_Logging_Agent etc.). Ensure correct agent names are used.
4.  For email tasks, separate body generation and saving into distinct steps if applicable (though Orchestrator might handle this).
.  **For scheduling, plan a step for Scheduling_Calendar_Agent to call the 'add_scheduled_event' tool with necessary arguments (subject, attendees as a list, time_preference as string, duration_minutes as integer).**
6.  **Your ONLY output MUST be the plan itself, prefixed with 'Plan:'.**
    Example: 'Plan: 1. Web_Search_Agent (...). 2. Summarization_Agent (...). 3a. Email_Composition_Sentiment_Agent (...). 3b. Email_Composition_Sentiment_Agent (...). 4. To_Do_List_Agent (...). **5. Scheduling_Calendar_Agent (action=call_tool, tool_name='add_scheduled_event', args={'subject':'Discuss CrewAI', 'attendees':['me', 'colleague@example.com'], 'time_preference':'tomorrow afternoon', 'duration_minutes':30})**. 6. To_Do_List_Agent (action=view_list). 7. Analytics_Logging_Agent (...).'
7.  DO NOT execute any tasks yourself...""",
    description="Analyzes query, creates a step-by-step plan including explicit tool calls for scheduling, uses correct agent names, returns only the plan prefixed with 'Plan:'."
)

# --- Specialized Task Agents (with Tools) ---

# Web Search Agent
web_search_agent = AssistantAgent(
    name="Web_Search_Agent",
    model_client=model_client,
    tools=[perform_web_search],
    reflect_on_tool_use=True, # Can generate a brief text describing the tool result
    system_message="""You are the Web Search Agent. Your **ONLY** function is to execute web searches using the 'perform_web_search' tool when instructed by the Orchestrator.
1.  Receive a search query instruction from the Orchestrator.
2.  Call the 'perform_web_search' tool with the exact query and parameters provided.
3.  **Your ONLY output MUST be the raw search result string returned by the tool.** If `reflect_on_tool_use` is enabled, you may preface it with a very brief statement like "Here are the search results:", but nothing more.
4.  **DO NOT** summarize, analyze, interpret the results, draft emails, add todos, or ask follow-up questions. Report ONLY the tool's direct output.""",
    description="STRICTLY performs web searches using 'perform_web_search' tool and reports ONLY the raw tool output."
)

# Summarization Agent
summarization_agent = AssistantAgent(
    name="Summarization_Agent",
    model_client=model_client,
    system_message="""You are the Summarization Agent. Your **ONLY** task is to summarize text provided by the Orchestrator.
1.  Expect to receive a **string containing text** from the Orchestrator. This text might be raw web search results including titles, snippets, and potentially links.
2.  Focus on the main textual content (especially snippets) to identify key points.
3.  Generate a concise summary strictly following the format and length instructions given by the Orchestrator (e.g., 'summarize in 3 bullet points').
4.  **Your ONLY output MUST be the generated summary text.**
5.  If you cannot process the input or understand the instructions, respond with a clear error message stating the problem (e.g., 'Error: Input text unclear', 'Error: Invalid format instruction').
6.  DO NOT perform web searches, draft emails, or call any tools.""",
    description="Summarizes provided text (potentially raw search results) based on instructions, returning ONLY the summary or a specific error."
)

# Scheduling and Calendar Agent
scheduling_calendar_agent = AssistantAgent(
    name="Scheduling_Calendar_Agent",
    model_client=model_client,
    tools=[add_scheduled_event], # Uses only this tool
    reflect_on_tool_use=True, # Can print the tool result
    system_message="""You are the Scheduling & Calendar Agent. Your **ONLY** task is to log simulated calendar events using the 'add_scheduled_event' tool when instructed by the Orchestrator.
1.  Receive scheduling details (subject, attendees list, time_preference string, duration_minutes integer) from the Orchestrator.
2.  Call the 'add_scheduled_event' tool using **EXACTLY** these details as arguments.
3.  Report **ONLY** the confirmation message returned by the tool.""",
    description="Logs simulated calendar events to a file using the 'add_scheduled_event' tool ONLY when instructed."
)

# Email Composition and Sentiment Analysis Agent
email_sentiment_agent = AssistantAgent(
    name="Email_Composition_Sentiment_Agent",
    model_client=model_client,
    tools=[save_email_draft],
    reflect_on_tool_use=True, # Can print the tool result
    system_message="""You are the Email Composition & Sentiment Analysis Agent. You have two distinct modes triggered by the Orchestrator's specific instructions:

**MODE 1: Draft Email Body**
1.  Instruction received: "Draft an email body..." with context, recipient, subject.
2.  Generate *only* the email **BODY** content based on the context.
3.  **Output ONLY the generated email body text.**
4.  DO NOT mention saving or call any tools in this mode.

**MODE 2: Save Email Draft (Tool Call)**
1.  Instruction received: "Call the 'save_email_draft' tool..." with explicit 'to', 'subject', and 'body' arguments provided by the Orchestrator.
2.  Call the 'save_email_draft' tool using *exactly* the provided arguments.
3.  Report **only** the confirmation message returned by the tool (e.g., "Email draft for... saved to ...").

**MODE 3: Analyze Sentiment** [Previous instructions remain]

**IMPORTANT:** Adhere strictly to the mode implied by the Orchestrator's instruction for each turn. Do not mix modes. Wait for the explicit 'Save Email Draft' instruction before attempting to call the tool.""",
    description="Mode 1: Generates email BODY. Mode 2: Calls 'save_email_draft' tool ONLY when explicitly instructed with full details. Mode 3: Analyzes sentiment."
)

# To-Do List Agent
todo_list_agent = AssistantAgent(
    name="To_Do_List_Agent",
    model_client=model_client,
    tools=[add_todo_item],
    reflect_on_tool_use=True, # Can print the tool result
    system_message="""You are the To-Do List Agent. Your **ONLY** task is to add items to the to-do list using the 'add_todo_item' tool when instructed by the Orchestrator.
1.  Receive the exact task description from the Orchestrator.
2.  Call the 'add_todo_item' tool with that exact task description.
3.  Report **only** the confirmation message returned by the tool.""",
    description="Adds tasks to the to-do list using 'add_todo_item' tool ONLY when instructed, reporting only the tool's result."
)

# --- Support Agents ---

# Follow-Up and Clarification Agent
follow_up_clarification_agent = AssistantAgent(
    name="Follow_Up_Clarification_Agent",
    model_client=model_client,
    system_message="""You are the Follow-Up & Clarification Agent.
1.  Receive instructions from the Orchestrator when clarification is needed.
2.  Formulate a concise question for the user based *only* on the Orchestrator's request.
3.  **Your ONLY output MUST be the question itself.**
    Example: 'What is the email address for Alex?'
4.  DO NOT add conversational filler.""",
    description="Formulates clarifying questions based ONLY on Orchestrator's request, returning ONLY the question."
)

# Analytics and Logging Agent
analytics_logging_agent = AssistantAgent(
    name="Analytics_Logging_Agent",
    model_client=model_client,
    tools=[log_event],
    reflect_on_tool_use=True, # Can print the tool result
    system_message="""You are the Analytics & Logging Agent. Your **ONLY** task is to log events using the 'log_event' tool when instructed by the Orchestrator.
1.  Receive the exact event details to log from the Orchestrator.
2.  Call the 'log_event' tool with those exact details.
3.  Report **only** the success message returned by the tool.""",
    description="Logs specific events using the 'log_event' tool ONLY when instructed, reporting only the tool's result."
)

# --- Group Chat Setup ---

agents = [
    user_proxy, orchestrator, input_preprocessor, intent_task_router,
    web_search_agent, summarization_agent, scheduling_calendar_agent,
    email_sentiment_agent, todo_list_agent, follow_up_clarification_agent,
    analytics_logging_agent,
]

# Termination Conditions
text_mention_termination = TextMentionTermination(
    text="TERMINATE",
    sources=[orchestrator.name]
)
max_messages_termination = MaxMessageTermination(max_messages=50)

# Combine conditions with OR (|)
combined_termination_condition = text_mention_termination | max_messages_termination

# Prompt to guide the LLM in selecting the next speaker
selector_prompt = """You are overseeing a conversation between agents working towards a user goal, following a plan.
Available roles:
{roles}

Conversation history:
{history}

Participants: {participants}.
The **Orchestrator** agent MUST manage the workflow step-by-step according to the initial plan. It delegates one task at a time and processes the result before delegating the next step.

Read the history carefully. Select the *next* agent to speak based on these STRICT rules:
1.  If the Orchestrator just gave a clear instruction/delegation to a *specific* agent, **SELECT THAT AGENT**.
2.  If *any* other agent (e.g., Web_Search_Agent, Summarization_Agent, Email_Composition_Sentiment_Agent, To_Do_List_Agent) just finished its task and reported its result (or reported an error), **SELECT THE Orchestrator**. The Orchestrator MUST process the result/error and decide the next step according to the plan (retry, move on, clarify). **DO NOT** select another specialized agent immediately after one has reported.
3.  If the Orchestrator just reported the "FINAL_OUTPUT:", **SELECT THE Orchestrator** again so it can say "TERMINATE".
4.  If the conversation seems stuck, unclear, or needs the next step from the plan, **SELECT THE Orchestrator**.
5.  If the Orchestrator instructed the Follow_Up_Clarification_Agent, select it. If Follow_Up_Clarification_Agent just spoke, select Orchestrator.
6.  If the User_Proxy just provided the initial request, select Orchestrator (or Input_Preprocessor if that's the first step).

**Only return the exact name of the single agent selected.** No extra text.
"""


# Create the SelectorGroupChat Team
team = SelectorGroupChat(
    participants=agents,
    model_client=model_client,
    termination_condition=combined_termination_condition,
    selector_prompt=selector_prompt,   
    allow_repeated_speaker=True, 
)

# --- Start the Chat ---

user_query = (
    "Research the key features of the CrewAI framework for multi-agent systems. "
    "Summarize the top 3 features in bullet points. "
    "Draft an email to my manager (manager@example.com) summarizing these features and suggesting we evaluate it. "
    "Also, add 'Schedule a meeting to discuss CrewAI evaluation' to my to-do list "
    "and try to schedule a 30-minute meeting for this discussion with me and 'colleague@example.com' for tomorrow afternoon using the calendar agent. "
    "Finally, log the completion of this entire request."
)

print(f"--- Starting Chat --- \nQuery: {user_query}\n{'='*30}")

async def run_team_chat_stream():
    """Runs the team chat as a stream and processes the result."""

    stream = team.run_stream(task=user_query)
    final_result = await Console(stream) 

    print(f"\n{'='*30}\n--- Chat Ended ---")
    
    if final_result:
        print(f"Termination Reason: {final_result.stop_reason}")
        print(f"Total Messages: {len(final_result.messages)}")
        messages = final_result.messages
    else:
        print("Chat result could not be obtained.")
        messages = [] # Empty list in case of error

    # --- Process Final Output  ---

    if messages:
        final_output = "Final output could not be processed (FINAL_OUTPUT: tag not found)." 
        log_summary = []

        for msg in reversed(messages): # Check messages from the end
            speaker_name: Optional[str] = None
            speaker_ref = getattr(msg, 'source', None)
            if hasattr(speaker_ref, 'name'): speaker_name = speaker_ref.name
            elif isinstance(speaker_ref, str): speaker_name = speaker_ref

            content_obj = getattr(msg, 'content', None)
            content_str = ""
            if isinstance(content_obj, str):
                content_str = content_obj.strip() # Trim whitespace
            elif content_obj is not None:
                content_str = str(content_obj).strip() # Convert to string and trim

            # Look for the Orchestrator's final message
            if speaker_name == orchestrator.name and "FINAL_OUTPUT:" in content_str:
                 try:
                     start_index = content_str.index("FINAL_OUTPUT:") + len("FINAL_OUTPUT:")
                     final_output_content = content_str[start_index:].strip()
                     if final_output_content.endswith("TERMINATE"):
                         final_output_content = final_output_content[:-len("TERMINATE")].strip()
                     final_output = final_output_content
                     # break # Finding the first might be enough, but checking all messages could be safer

                 except ValueError:
                     pass # Continue if 'FINAL_OUTPUT:' tag not found

            # Collect log messages
            elif speaker_name == analytics_logging_agent.name and "logged successfully" in content_str:
                 log_summary.append(content_str)

    print(f"\n--- Final Output ---")
    print(final_output)

    print("\n--- Logged Events Summary ---")
    if log_summary:
        for log_entry in reversed(log_summary): # Reverse for chronological order
            print(f"- {log_entry}")
    else:
        print("No logged events found.")

    # Print confirmation of generated files
    if os.path.exists("assistant_log.txt"):
        print("\nLog details written to 'assistant_log.txt'.")
    if os.path.exists("calendar_log.txt"):
        print("Simulated calendar events written to 'calendar_log.txt'.")
    if os.path.exists("email_drafts.txt"):
        print("Email drafts saved to 'email_drafts.txt'.")
    if os.path.exists("todo_list.txt"):
        print("To-Do items saved to 'todo_list.txt'.")


# Run the asynchronous chat
if __name__ == "__main__":
    try:
        # Call run_team_chat_stream with await (for Jupyter/IPython or async context)
        await run_team_chat_stream()
    except Exception as e:
        print(f"\n--- Runtime Error ---")
        import traceback
        traceback.print_exc()

--- Starting Chat --- 
Query: Research the key features of the CrewAI framework for multi-agent systems. Summarize the top 3 features in bullet points. Draft an email to my manager (manager@example.com) summarizing these features and suggesting we evaluate it. Also, add 'Schedule a meeting to discuss CrewAI evaluation' to my to-do list and try to schedule a 30-minute meeting for this discussion with me and 'colleague@example.com' for tomorrow afternoon using the calendar agent. Finally, log the completion of this entire request.
---------- user ----------
Research the key features of the CrewAI framework for multi-agent systems. Summarize the top 3 features in bullet points. Draft an email to my manager (manager@example.com) summarizing these features and suggesting we evaluate it. Also, add 'Schedule a meeting to discuss CrewAI evaluation' to my to-do list and try to schedule a 30-minute meeting for this discussion with me and 'colleague@example.com' for tomorrow afternoon using the c